# A simple RAG application using open-source models

In [3]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"OpenAI API Key: {OPENAI_API_KEY}")

# List models
models = openai.models.list()

for model in models.data:
  print(f"Model Name: {model.id}")

# MODEL = "gpt-3.5-turbo"
# MODEL = "gpt-4o"
MODEL = "llama3"   # the best model 


OpenAI API Key: None


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [9]:
import ollama
response = ollama.chat(model='llama3:8b', messages=[
    {
        'role': 'user',
        'content': 'Why is the sky blue?'
    },
])
print(response['message']['content'])

The color of the sky can appear to change depending on the time of day, atmospheric conditions, and the amount of dust or pollution in the air. However, under normal circumstances, the sky typically appears blue because of a phenomenon called Rayleigh scattering.

Rayleigh scattering is the scattering of light by small particles or molecules in the atmosphere, such as nitrogen (N2), oxygen (O2), and aerosols. When sunlight enters Earth's atmosphere, it encounters these tiny particles and scatters in all directions.

Here's what happens:

1. **Short wavelengths scatter more**: The shorter wavelengths of visible light, like blue and violet, are scattered more than the longer wavelengths, like red and orange. This is because the smaller particles in the atmosphere are more effective at scattering shorter wavelengths.
2. **Blue light dominates**: As a result of this scattering, the blue light becomes distributed throughout the sky, making it appear blue to our eyes.
3. **Red light continue

KeyError: 'messages'

In [5]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

MODEL = "llama3" 
if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    # each model has its own embeddings
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh! Do you want to hear another?"

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that brought a smile to your face!"

In [62]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

# and here we test
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [63]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's my name'?"})

'Your name is Santiago.'

In [64]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [65]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-readymachine learning\nsystems from the ground up.\nNext cohort:\xa0July1 - 18, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic ideas\nfor the hundredth time. Most people are stuck in beginner mode, and finding help to\nsolve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models for\nsome of the largest companies in the world. I want to show you how to do the same."This is the best machine learning course I\'ve done.\nWorth every cent."\n— Jose Reyes, AI/ML at Cevo Australia5/19/24, 3:28 PM Building Machine Learning Systems That Don\'t Suck\nh

In [66]:
from langchain_community.vectorstores import DocArrayInMemorySearch

# here we generate a vector store in memeory and provide the corresponding embedding
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [67]:
# here we create a retriever for our vector store
retriever = vectorstore.as_retriever()

# retriever should return 4 top documents that are most relevant from the vector store.
retriever.invoke("machine learning")

[Document(page_content="Here is a summary of what makes this program unique:\nYou'll design and write the code to build an end-to-end machine learning\nsystem starting from scratch.\nYou'll learn best practices to tackle the most significant challenges machine\nlearning engineers face to build, evaluate, run, monitor, and maintain machine\nlearning systems in real-world scenarios.\nYou'll learn how to use techniques like active learning, distributed training,\nadversarial validation, human-in-the-loop deployments, model compression,\ntest-time augmentation, testing in production, among many others.\nYou'll learn how to create training, deploying, monitoring, and inference\npipelines using Amazon SageMaker and open-source tools.\nForget about theoretical concepts. This program will show you some of the things I've\nlearned from real-life examples I've built during more than 30 years in the industry.\nCheck the program syllabus\xa0→\nReal-life examples and case studies\nLearn from practi

In [68]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,  # here we take a question and pass it to the retriever  using the pipe operator
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [69]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

# looping throw the questions
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")  # passing it to the chain
    print()

Question: What is the purpose of the course?
Answer: According to the context, the purpose of the course is to "unlearn what you think machine learning is" and help learners build practical skills in building machine learning systems that work in the real world. The course promises to provide hands-on training through 18 hours of live interactive sessions, step-by-step coding instructions, a final project with feedback, and access to a community of learners.

Question: How many hours of live sessions?
Answer: According to the context, there are 18 hours of live, interactive sessions in the program.

Question: How many coding assignments are there in the program?
Answer: According to the context, there are 100 coding assignments and practice questions in the program.

Question: Is there a program certificate upon completion?
Answer: Yes, according to the context, there is a program certificate upon completion.

Question: What programming language will be used in the program?
Answer: The

In [70]:
# thsi si another way to loop . each one goes in parallel behind the scene, but it wil display all of them at onces 
# when all answers are ready
chain.batch([{"question": q} for q in questions])

['According to the context, the purpose of the course is "to help you unlearn what you think machine learning is" and to learn from years of experience and real-world examples how to build practical, end-to-end machine learning systems. The course aims to provide hands-on training and teach students how to solve real-world problems using machine learning.',
 'According to the context, the program has 18 hours of live, interactive sessions.',
 'According to the context, there are "100 coding assignments and practice questions" in the program.',
 'Yes, according to the context, there is a program certificate upon completion.',
 'Based on the context, the answer to this question is:\n\nPython\n\nAccording to the document, "You have experience writing code. We\'ll use Python throughout the class, but you won\'t have any problems if you know any other language." This indicates that Python will be the primary programming language used in the program.',
 'The answer to this question can be fo

In [71]:
# this is  interesting one as it will stream answer (like show one character after another)
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s, end="", flush=True)

Based on the context, the purpose of the course is to help students "unlearn what you think machine learning is" and learn practical skills in building machine learning systems that work in the real world. The course aims to provide hands-on experience, step-by-step coding instructions, and a final project where students can build a complete solution and receive direct feedback on their work.

In [109]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents

[Document(page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done ever. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t SuckA live, interactive program that\'ll help you build production-ready machine learning systems from the ground up.Next cohort:\xa0April 8 - 25, 2024Check the schedule for more details about upcoming cohorts.Register nowLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve real-world problems is hard.I want to change that.I started writing software 30 years ago. I\'ve written pipelines and trained models for some of the largest companies in the world. I want to show you how to do the same.This is the class I wish I had taken when I star